<a href="https://colab.research.google.com/github/HungLV2512/NLP/blob/main/ASM2/CBOW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
lines = []
with open('/content/drive/MyDrive/Colab Notebooks/NLP/truyen_kieu_data.txt', 'r', encoding='utf-8') as file:
    lines = file.readlines()

In [ ]:
import string
import numpy as np
from tensorflow.keras.layers import Embedding, Lambda, Dense
import tensorflow.keras.backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [ ]:
PUNCT_TO_REMOVE = string.punctuation + string.digits + '\n'
def clean_text(text):
    text = text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))
    text = text.lower()
    return text
clean_text(lines[0])

'trăm năm trong cõi người ta'

In [ ]:
corpus = [clean_text(line) for line in lines]

In [ ]:
def get_centers_and_contexts(corpus, max_window_size=2):
    centers, contexts = [], []
    for line in corpus:
        line = line.split()

        if len(line) <= 2*max_window_size:
            continue

        for i in range(max_window_size, len(line) - max_window_size):
            centers.append(line[i])
            idxs = list(range(i-max_window_size, i+max_window_size+1))
            idxs.remove(i)
            contexts.append(" ".join([line[idx] for idx in idxs]))

    return centers, contexts

In [ ]:
centers, contexts = get_centers_and_contexts(corpus)
len(centers), len(contexts)

(9778, 9778)

In [ ]:
centers[:2], contexts[:2]

(['trong', 'cõi'], ['trăm năm cõi người', 'năm trong người ta'])

In [ ]:
max_length = 4
embedding_size = 200

In [ ]:
tokenizer = Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts(corpus)

In [ ]:
vocab_size = len(tokenizer.word_index) + 1

In [ ]:
train_seq = tokenizer.texts_to_sequences(contexts)
train_seq_pad = pad_sequences(train_seq, maxlen=max_length, truncating='post', padding='post')

In [ ]:
train_labels = [to_categorical(tokenizer.word_index[label], len(tokenizer.word_index)+1) for label in centers]
train_labels = np.array(train_labels)

In [ ]:
cbow = Sequential()
cbow.add(Embedding(vocab_size, embedding_size, input_length=max_length))
cbow.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(embedding_size,)))
cbow.add(Dense(vocab_size, activation='softmax'))
cbow.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 4, 200)            482400    
                                                                 
 lambda (Lambda)             (None, 200)               0         
                                                                 
 dense (Dense)               (None, 2412)              484812    
                                                                 
Total params: 967212 (3.69 MB)
Trainable params: 967212 (3.69 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
cbow.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
cbow.fit(train_seq_pad, train_labels, epochs=50, verbose=1)

Epoch 1/50
306/306 [==============================] - 10s 26ms/step - loss: 7.4531 - acc: 0.0180
Epoch 2/50
306/306 [==============================] - 3s 9ms/step - loss: 6.7261 - acc: 0.0204
Epoch 3/50
306/306 [==============================] - 3s 8ms/step - loss: 6.5654 - acc: 0.0226
Epoch 4/50
306/306 [==============================] - 2s 8ms/step - loss: 6.3883 - acc: 0.0351
Epoch 5/50
306/306 [==============================] - 3s 8ms/step - loss: 6.1367 - acc: 0.0553
Epoch 6/50
306/306 [==============================] - 2s 6ms/step - loss: 5.8040 - acc: 0.0811
Epoch 7/50
306/306 [==============================] - 2s 7ms/step - loss: 5.4006 - acc: 0.1136
Epoch 8/50
306/306 [==============================] - 2s 7ms/step - loss: 4.9469 - acc: 0.1556
Epoch 9/50
306/306 [==============================] - 2s 7ms/step - loss: 4.4628 - acc: 0.2105
Epoch 10/50
306/306 [==============================] - 2s 6ms/step - loss: 3.9748 - acc: 0.2763
Epoch 11/50
306/306 [==========================

In [ ]:
sample_text = 'trăm năm cõi người'
sample_seq = tokenizer.texts_to_sequences([sample_text])
sample_seq_pad = pad_sequences(sample_seq, maxlen=max_length, truncating='post', padding='post')
cbow.predict(sample_seq_pad)

1/1 [==============================] - 0s 94ms/step


array([[4.66419235e-14, 4.60987009e-14, 1.53871442e-04, ...,
        5.34828395e-08, 4.86869659e-14, 1.07789866e-07]], dtype=float32)

In [ ]:
tokenizer.index_word[np.argmax(cbow.predict(sample_seq_pad))]

1/1 [==============================] - 0s 40ms/step


'trong'